In [2]:
from benchmark import Unit
from sys.intrinsics import strided_load
from math import CeilDivableRaising
from memory import memset_zero
from memory.unsafe import DTypePointer
from random import rand, random_float64
from sys.info import simdwidthof
from runtime.llcl import Runtime
from algorithm import parallelize
import time


In [3]:
alias type = DType.float32

struct Matrix[rows: Int, cols: Int]:
    var data: DTypePointer[type]

    # Initialize zeroeing all values
    fn __init__(inout self):
        self.data = DTypePointer[type].alloc(rows * cols)
        memset_zero(self.data, rows * cols)

    # Initialize taking a pointer, don't set any elements
    fn __init__(inout self, data: DTypePointer[type]):
        self.data = data

    # Initialize with random values
    @staticmethod
    fn rand() -> Self:
        var data = DTypePointer[type].alloc(rows * cols)
        rand(data, rows * cols)
        return Self(data)

    fn __getitem__(self, y: Int, x: Int) -> Scalar[type]:
        return self.load[1](y, x)

    fn __setitem__(self, y: Int, x: Int, val: Scalar[type]):
        self.store[1](y, x, val)

    fn load[nelts: Int](self, y: Int, x: Int) -> SIMD[type, nelts]:
        return self.data.load[width=nelts](y * self.cols + x)

    fn store[nelts: Int](self, y: Int, x: Int, val: SIMD[type, nelts]):
        return self.data.store[width=nelts](y * self.cols + x, val)

In [4]:
alias nelts = simdwidthof[DType.float32]() * 2

In [5]:
from algorithm import vectorize
from algorithm import parallelize

#write similar code for fn matadd_parallelized
fn matadd_parallelized(C: Matrix, A: Matrix, B: Matrix):
    @parameter
    fn calc_row(m: Int):
        for n in range(A.cols):
            @parameter
            fn add[nelts : Int](n : Int):
                C.store[nelts](m,n, A[m,n] + B[m,n])
            vectorize[add, nelts, size = C.cols]()
    parallelize[calc_row](C.rows, C.rows)
fn matsub_parallelized(C: Matrix, A: Matrix, B: Matrix):
    @parameter
    fn calc_row(m: Int):
        for n in range(A.cols):
            @parameter
            fn sub[nelts : Int](n : Int):
                C.store[nelts](m,n, A[m,n] - B[m,n])
            vectorize[sub, nelts, size = C.cols]()
    parallelize[calc_row](C.rows, C.rows)


    

In [6]:
alias M = 128
alias N = 128
alias K = 128

@always_inline
fn bench[
    func: fn (Matrix, Matrix, Matrix) -> None](base_gflops: Float64):
    var C = Matrix[M, N]()
    var A = Matrix[M, K].rand()
    var B = Matrix[K, N].rand()

    @always_inline
    @parameter
    fn test_fn():
        _ = func(C, A, B)

    var secs = benchmark.run[test_fn](max_runtime_secs=1).mean()

    A.data.free()
    B.data.free()
    C.data.free()
#    # also print all the matrices
#    print("Matrix A:")
#    for i in range(M):
#        for j in range(K):
#            print(A[i,j], " ")
#        print("\n")
#    print("Matrix B:")
#    for i in range(K):
#        for j in range(N):
#            print(B[i,j], " ")
#        print("\n")
#    print("Matrix C:")
#    for i in range(M):
#        for j in range(N):
#            print(C[i,j], " ")
#        print("\n")
#
    var gflops = ((2 * M * N * K) / secs) / 1e9

    print(gflops, "GFLOP/s")

In [7]:
bench[matadd_parallelized](0.0019909878018114003)
bench[matsub_parallelized](0.0019909878018114003)

45.317157286412858 GFLOP/s
44.059655981981635 GFLOP/s
